In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
method_tags = ['NC', 'DS', 'DS+MRH', 'DSX', 'DSX+MRH']
df = pd.read_csv('out/compare/benchmark/numer_results.csv')
for m in method_tags[1:]:
    df['prc_' + m] = (-df[m]+df['NC'])/(df['NC'])*100
    df['prc_' + m] = df['prc_' + m].round(decimals=2)
df

,parameter,prc_cancel,NC,DS,DS+MRH,DSX,DSX+MRH,prc_DS,prc_DS+MRH,prc_DSX,prc_DSX+MRH
0,cv_h,0,0.526201,0.529697,0.459051,0.510019,0.447442,-0.66,12.76,3.08,14.97
1,cv_h,12,0.635314,0.531481,0.482580,0.514827,0.466591,16.34,24.04,18.97,26.56
2,cv_h,25,0.758590,0.606335,0.560532,0.589301,0.566277,20.07,26.11,22.32,25.35
3,wt,0,3.283640,3.280683,3.072857,3.265405,3.055152,0.09,6.42,0.56,6.96
4,wt,12,4.410861,3.974714,3.740851,3.892873,3.667786,9.89,15.19,11.74,16.85
5,wt,25,6.347503,5.433980,5.087219,5.452367,5.235302,14.39,19.85,14.10,17.52
6,rbt,0,5.802635,5.906558,5.302747,5.786011,5.426725,-1.79,8.61,0.29,6.48
7,rbt,12,9.092081,7.300763,7.173659,7.255184,7.040866,19.70,21.10,20.20,22.56
8,rbt,25,14.583704,11.629561,10.566865,11.702990,11.221716,20.26,27.54,19.75,23.05
9,95load,0,47.000000,48.000000,46.450000,46.000000,44.000000,-2.13,1.17,2.13,6.38


In [12]:
method_tags = ['NC53', 'NC80', 'DS+MRH53', 'DS+MRH80', 'DSX+MRH53', 'DSX+MRH80']
df = pd.read_csv('out/compare/infinite capacity/numer_results.csv')
for m in method_tags[1:]:
    df['prc_' + m] = (-df[m]+df['NC53'])/(df['NC53'])*100
    df['prc_' + m] = df['prc_' + m].round(decimals=2)
df

,parameter,prc_cancel,NC53,NC80,DS+MRH53,DS+MRH80,DSX+MRH53,DSX+MRH80,prc_NC80,prc_DS+MRH53,prc_DS+MRH80,prc_DSX+MRH53,prc_DSX+MRH80
0,cv_h,12,0.635314,0.645532,0.482580,0.485673,0.466591,0.467891,-1.61,24.04,23.55,26.56,26.35
1,cv_h,25,0.758590,0.813602,0.560532,0.570365,0.566277,0.559839,-7.25,26.11,24.81,25.35,26.20
2,wt,12,4.410861,4.352093,3.740851,3.671241,3.667786,3.789103,1.33,15.19,16.77,16.85,14.10
3,wt,25,6.347503,6.211394,5.087219,4.796748,5.235302,4.912688,2.14,19.85,24.43,17.52,22.60
4,rbt,12,9.092081,9.085797,7.173659,6.868992,7.040866,7.171299,0.07,21.10,24.45,22.56,21.13
5,rbt,25,14.583704,14.117582,10.566865,10.355100,11.221716,10.553066,3.20,27.54,29.00,23.05,27.64
6,95load,12,53.000000,61.000000,53.000000,61.000000,52.150000,55.000000,-15.09,0.00,-15.09,1.60,-3.77
7,95load,25,53.000000,76.000000,53.000000,68.000000,53.000000,67.200000,-43.40,0.00,-28.30,0.00,-26.79
8,50load,12,25.000000,23.000000,25.000000,25.000000,26.000000,25.000000,8.00,0.00,0.00,-4.00,0.00
9,50load,25,31.000000,26.000000,32.000000,31.000000,31.000000,29.500000,16.13,-3.23,0.00,0.00,4.84


In [19]:
tdf80 = pd.read_pickle('out/NC/0729-011457-trip_record_ob.pkl')
tdf53 = pd.read_pickle('out/NC/0729-011546-trip_record_ob.pkl')
pdf80 = pd.read_pickle('out/NC/0729-011457-pax_record_ob.pkl')
pdf53 = pd.read_pickle('out/NC/0729-011546-pax_record_ob.pkl')
denied80 = pdf80[pdf80['denied'] == 1].copy()
denied53 = pdf53[pdf53['denied'] == 1].copy()
notdenied80 = pdf80[pdf80['denied'] == 0].copy()
notdenied53 = pdf53[pdf53['denied'] == 0].copy()

In [35]:
def wtime(t0, t1, df):
    tmp_df = df[(df['arr_time'] >= t0) & (df['arr_time'] <= t1)].copy()
    wt = tmp_df['board_time'] - tmp_df['arr_time']
    return round(wt.mean(), 1)
def ratio(t0, t1, df):
    tmp_df = df[(df['arr_time'] >= t0) & (df['arr_time'] <= t1)].copy()
    denied = tmp_df[tmp_df['denied'] == 1].shape[0]
    tot = tmp_df.shape[0]
    return str(round(denied/tot*100))+'%'
def wtime_pctile(t0, t1, df, pctile=0.95):
    tmp_df = df[(df['arr_time'] >= t0) & (df['arr_time'] <= t1)].copy()
    wt = tmp_df['board_time'] - tmp_df['arr_time']
    return round(wt.quantile(pctile), 1)
def wtime_subset(t0, t1, df, stops, o_idx, d_idx):
    tmp_df = df[(df['arr_time'] >= t0) & (df['arr_time'] <= t1)].copy()
    d_stops = {s: stops.index(s) for s in stops}
    tmp_df = tmp_df.replace({'orig_idx':d_stops, 'dest_idx':d_stops})
    tmp_df = tmp_df[(tmp_df['orig_idx']>=o_idx) & (tmp_df['dest_idx']<=d_idx)]
    wt = tmp_df['board_time'] - tmp_df['arr_time']
    return round(wt.mean(), 1)

In [180]:
ts = [6.0*60*60, 8.0*60*60]
ts2 = [6.0*60*60, 7.0*60*60]
ts3 = [7.0*60*60, 8.0*60*60]
for t in (ts, ts2, ts3):
    print('----')
    print(t)
    print(f'capacity 53 with ratio {ratio(*t, pdf53)}')
    print(f'all {wtime(*t, pdf53)} denied {wtime(*t, denied53)} and not denied {wtime(*t, notdenied53)}')
    print(f'capacity 80 with ratio {ratio(*t, pdf80)}')
    print(f'all {wtime(*t, pdf80)} denied {wtime(*t, denied80)} and not denied {wtime(*t, notdenied80)}')

----
[21600.0, 28800.0]
capacity 53 with ratio 6%
all 383.8 denied 966.5 and not denied 349.8
capacity 80 with ratio 1%
all 381.1 denied 1119.3 and not denied 372.4
----
[21600.0, 25200.0]
capacity 53 with ratio 3%
all 459.4 denied 1169.0 and not denied 438.8
capacity 80 with ratio 0%
all 453.6 denied 1853.1 and not denied 451.2
----
[25200.0, 28800.0]
capacity 53 with ratio 7%
all 354.8 denied 933.0 and not denied 314.2
capacity 80 with ratio 2%
all 353.2 denied 1087.0 and not denied 341.7


In [45]:
from Inputs import STOPS_OUT_FULL_PATT
scenarios = [['NC/0729-011407', 'NC/0729-011546'],
             ['NC/0729-011316', 'NC/0729-011457'],
             ['DS+MRH/0729-011427', 'DS+MRH/0729-011604'],
             ['DS+MRH/0729-011337', 'DS+MRH/0729-011517'],
             ['DSX+MRH/0729-011447', 'DSX+MRH/0729-011623'],
             ['DSX+MRH/0729-011357', 'DSX+MRH/0729-011536']]
method_tags = ['NC53', 'NC80', 'DS+MRH53', 'DS+MRH80', 'DSX+MRH53', 'DSX+MRH80']
wts = {m: [] for m in method_tags}
wts95 = {m: [] for m in method_tags}
ts = [6.5*60*60, 8.5*60*60]
for i in range(len(scenarios)):
    for j in range(len(scenarios[i])):
        df = pd.read_pickle('out/' + scenarios[i][j] + '-pax_record_ob.pkl')
        wts[method_tags[i]].append(wtime_pctile(*ts, df, pctile=0.95)/60)
        wts95[method_tags[i]].append(wtime_subset(*ts, df, STOPS_OUT_FULL_PATT, 49, 66)/60)
wts, wts95
# print(wtime_pctile(*ts, pdf53, pctile=0.95),wtime_pctile(*ts, pdf80,pctile=0.95))
# print(wtime_subset(*ts, pdf53, STOPS_OUT_FULL_PATT, 49, 66), wtime_subset(*ts, pdf80, STOPS_OUT_FULL_PATT, 49, 66))

({'NC53': [11.705, 19.573333333333334],
  'NC80': [11.868333333333334, 18.366666666666667],
  'DS+MRH53': [9.573333333333332, 14.331666666666667],
  'DS+MRH80': [9.146666666666667, 13.291666666666666],
  'DSX+MRH53': [9.2, 14.94],
  'DSX+MRH80': [9.421666666666665, 13.478333333333333]},
 {'NC53': [5.801666666666667, 8.45],
  'NC80': [5.376666666666667, 7.7716666666666665],
  'DS+MRH53': [4.8566666666666665, 6.595],
  'DS+MRH80': [4.503333333333333, 5.588333333333334],
  'DSX+MRH53': [4.446666666666667, 6.448333333333333],
  'DSX+MRH80': [4.395, 5.59]})

In [188]:
tdf53[(tdf53['trip_id'] == 911318020) & (tdf53['replication'] == 20) & (tdf53['stop_sequence'] >= 40)]

,bus_id,trip_id,stop_id,arr_sec,dep_sec,pax_load,ons,offs,denied,hold_time,skipped,schd_sec,stop_sequence,dist_traveled,expressed,replication
32969,897302,911318020,427,31616.161833,31627.561833,53,3,3,7,0,0,31008.0,40,26022,0,20
32973,897302,911318020,428,31665.534035,31679.515533,52,4,5,0,0,0,31058.0,41,26746,0,20
32979,897302,911318020,429,31728.522419,31735.422419,51,1,2,0,0,0,31101.0,42,27382,0,20
32983,897302,911318020,430,31757.254242,31773.254242,53,5,3,1,0,0,31155.0,43,28209,0,20
32987,897302,911318020,431,31830.294327,31830.294327,53,0,0,7,0,0,31202.0,44,28900,0,20
32993,897302,911318020,4727,31907.495602,31932.695602,53,9,9,3,0,0,31227.0,45,29212,0,20
32998,897302,911318020,432,31977.964925,31987.064925,53,2,2,8,0,0,31260.0,46,29648,0,20
33004,897302,911318020,15279,32048.255079,32048.255079,53,0,0,9,0,0,31344.0,47,30496,0,20
33006,897302,911318020,16049,32064.816417,32073.916417,53,2,2,14,0,0,31396.0,48,30941,0,20
33013,897302,911318020,435,32135.323687,32143.171072,53,1,1,18,0,0,31462.0,49,31668,0,20


In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
tdf = pd.read_csv('in/rt_81_2022-05/trips.txt')
df = pd.read_csv('in/rt_81_2022-05/avl.csv')
sdf = pd.read_csv('in/rt_81_2022-05/gtfs_stop_times.txt')

<ipython-input-61-fc332f19a5d7>:4: DtypeWarning: Columns (1,2,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv('in/rt_81_2022-05/trips.txt')


In [62]:
df['arr_sec'] = (pd.to_datetime(df['event_time'])- pd.to_datetime('2022-05-02 00:00:00')).dt.total_seconds()%86400
df['arr_sec'] = df['arr_sec'].round()
df['dep_sec'] = df['arr_sec'] + df['dwell_time']
df = df[df['trip_id'].notnull()].copy()
df = df[df['stop_id'].notnull()].copy()
df['trip_id'] = df['trip_id'].astype(int)
df['stop_id'] = df['stop_id'].astype(int).astype(str)
df = df.rename(columns={'event_time':'arr_time'})
df.to_csv('in/rt_81_2022-05/avl2.csv', index=False)